In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_key import gkey

# Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [2]:
 # Store csv created in part one into a DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new norfolk,-42.78,147.06,46.00,84,91,3.00,AU,1595348903
1,1,lagoa,39.05,-27.98,68.88,72,0,2.66,PT,1595348903
2,2,pangnirtung,66.15,-65.71,48.20,93,90,6.93,CA,1595348903
3,3,port macquarie,-31.43,152.92,48.99,88,89,7.67,AU,1595348903
4,4,korla,41.76,86.15,79.00,74,75,4.00,CN,1595348903


 Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [6]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

 Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [7]:
 # Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,senanga,-16.12,23.27,70.12,24,0,6.04,ZM,1595348912
209,209,boysun,38.21,67.20,79.25,17,0,8.12,UZ,1595348931
252,252,castro,-24.79,-50.01,72.34,51,0,8.52,BR,1595348722
330,330,sembakung,3.82,117.25,74.43,95,0,1.36,ID,1595348951
334,334,usinsk,65.99,57.53,71.06,55,0,7.00,RU,1595348952
376,376,buriti,-22.93,-45.39,78.80,39,0,2.24,BR,1595348959
435,435,raga,8.46,25.68,78.55,80,0,3.02,SS,1595348968
444,444,zabol,32.25,67.25,73.18,21,0,7.09,AF,1595348969
538,538,iskateley,67.68,53.15,71.01,59,0,7.90,RU,1595348987


#Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
69,senanga,ZM,-16.12,23.27,
209,boysun,UZ,38.21,67.20,
252,castro,BR,-24.79,-50.01,
330,sembakung,ID,3.82,117.25,
334,usinsk,RU,65.99,57.53,
376,buriti,BR,-22.93,-45.39,
435,raga,SS,8.46,25.68,
444,zabol,AF,32.25,67.25,
538,iskateley,RU,67.68,53.15,


In [10]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
69,senanga,ZM,-16.12,23.27,Christopher Nasilele House
209,boysun,UZ,38.21,67.20,Gaza Hotel
252,castro,BR,-24.79,-50.01,CHACARA BAILLY
330,sembakung,ID,3.82,117.25,
334,usinsk,RU,65.99,57.53,Dobro Pozhalovaty
376,buriti,BR,-22.93,-45.39,Pousada Sao Cristovao
435,raga,SS,8.46,25.68,
444,zabol,AF,32.25,67.25,
538,iskateley,RU,67.68,53.15,Hostel Agat


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))